In [687]:
rm(list = ls())

In [688]:
# libraries
suppressMessages({
  library(tidyverse)
  library(broom)
  library(ranger)
}
)

In [689]:
Data <- read.csv("/Users/mpaga/Downloads/train.csv",sep=",")

In [690]:
dim(Data)

[1] 1460   81

In [691]:
names(Data)

 [1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"  
 [5] "LotArea"       "Street"        "Alley"         "LotShape"     
 [9] "LandContour"   "Utilities"     "LotConfig"     "LandSlope"    
[13] "Neighborhood"  "Condition1"    "Condition2"    "BldgType"     
[17] "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"  
[25] "Exterior2nd"   "MasVnrType"    "MasVnrArea"    "ExterQual"    
[29] "ExterCond"     "Foundation"    "BsmtQual"      "BsmtCond"     
[33] "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"    "BsmtFinType2" 
[37] "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"    
[45] "X2ndFlrSF"     "LowQualFinSF"  "GrLivArea"     "BsmtFullBath" 
[49] "BsmtHalfBath"  "FullBath"      "HalfBath"      "BedroomAbvGr" 
[53] "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd"  "Functional"   
[57] "Fireplaces"    "FireplaceQu"

In [692]:
#remove Id colum
Data["Id"] <- NULL
dim(Data)

[1] 1460   80

In [693]:
# numeric data
numData <- Data |> 
  select_if(is.numeric) 

In [694]:
hist(Data$SalePrice)

In [695]:
summary(Data$SalePrice)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  34900  129975  163000  180921  214000  755000 

In [696]:
skimr::skim_without_charts(Data,where(is.numeric))-> num_skimData

In [697]:
(numVal_na <- num_skimData$skim_variable[num_skimData$n_missing !=0] )

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [698]:
skimr::skim_without_charts(Data,where(is.character))-> cat_skimData

In [699]:
summary(cat_skimData)

── Data Summary ────────────────────────
                           Values
Name                       Data  
Number of rows             1460  
Number of columns          80    
_______________________          
Column type frequency:           
  character                43    
________________________         
Group variables            None  

Let's impute some of these numerical features

In [700]:
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
0.177397260 0.005479452 0.055479452 

In [701]:
# list of numeric features to impute
imputeVal_list <- apply(Data[numVal_na],2, FUN = "median",na.rm = T,simplify = list)

In [702]:
#impute numerical features 
Data[numVal_na] <- replace_na(Data[numVal_na] ,
                              replace = imputeVal_list
)

In [703]:
# check  na
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
          0           0           0 

In [704]:
# duplicates rows
Data |> 
  select_if(is.numeric) |> 
  unique() |> 
  nrow()

[1] 1460

In [705]:
# select numeric features
Data |> 
  select_if(is.numeric)  ->numData 

# check relationship with target (linear model)
# h0 : there no relationship between X and Y(SalePrice)
# h1 :otherwise

  lm(SalePrice~.,numData) |> 
    summary() |> 
    tidy() |> 
    filter(p.value <=0.05) |> 
    nrow()

[1] 19

28 numerical features have predictive effect on target feature. 

In [706]:
# correlation in numeric features
for (feature in names(numData)[-1]){
  if (cor(numData[names(numData)][1],numData[feature]) >= 0.8 ) print(feature)
}

There is no carrelated features in numeric features 

cat var feat engineering  

In [707]:
# categorical features presenting missing values 
sum(cat_skimData$n_missing  > 0)

[1] 16

In [708]:
# n_missing proportion > 0.95 in cat features 
cat_rid <- cat_skimData$skim_variable[cat_skimData$n_missing >= 0.95] 
length(cat_rid)

[1] 16

16 features present missing values or NA proportion above 0.95, we could get rid of it  

In [709]:
# remove these categorical features
Data[,cat_rid]<- NULL

In [710]:
# check categorical features with NA in Data 
sum(cat_rid %in% names(Data))
rm(cat_rid)

In [711]:
# new data dimension
dim(Data)

[1] 1460   64

In [712]:
# NA check
colMeans(is.na(Data)) |> sum()

[1] 0

There is no longer NA in this dataset

In [713]:
summary(lm(SalePrice~. ,Data)) |> 
  tidy() |> 
  filter(p.value <=0.05) |> 
  print(n=10)

# A tibble: 59 × 5
   term              estimate std.error statistic  p.value
# A tibble: 59 × 5
   term              estimate std.error statistic  p.value


   <chr>                <dbl>     <dbl>     <dbl>    <dbl>
 1 MSZoningFV       33495.    12123.         2.76 5.81e- 3
 2 MSZoningRH       25399.    12189.         2.08 3.74e- 2
 3 MSZoningRL       27888.    10386.         2.69 7.35e- 3
 4 MSZoningRM       26270.     9716.         2.70 6.95e- 3
 5 LotFrontage         96.2      44.0        2.19 2.89e- 2
 6 LotArea              0.670     0.109      6.16 9.47e-10
 7 StreetPave       36874.    12184.         3.03 2.52e- 3
 8 LandContourHLS   13435.     5242.         2.56 1.05e- 2
 9 LandContourLvl    7446.     3775.         1.97 4.88e- 2
10 LotConfigCulDSac  9864.     3407.         2.90 3.86e- 3
# ℹ 49 more rows
# ℹ Use `print(n = ...)` to see more rows
   <chr>                <dbl>     <dbl>     <dbl>    <dbl>
 1 MSZoningFV       33495.    12123.         2.76 5.81e- 3
 2 MSZoningRH       25399.    12189.         2.08 3.74e- 2
 3 MSZoningRL       27888.    10386.         2.69 7.35e- 3
 4 MSZoningRM       26270.     9716.         2.70 6.95e-

In [714]:
ranger(SalePrice~.,Data,
  importance = 'impurity') |> 
  importance() |> 
  sort(decreasing = T) |> 
  tidy() |> 
  mutate(proportion= x/sum(x)) |> 
  filter(proportion >= 0.05)

Warning message:
In tidy.numeric(sort(importance(ranger(SalePrice ~ ., Data, importance = "impurity")),  :
  'tidy.numeric' is deprecated.
See help("Deprecated")


# A tibble: 8 × 3
  names             x proportion
  <chr>         <dbl>      <dbl>
1 OverallQual 1.16e12     0.128 
2 GrLivArea   8.07e11     0.0888
3 GarageCars  6.16e11     0.0678
4 TotalBsmtSF 5.96e11     0.0655
5 GarageArea  5.37e11     0.0591
6 X1stFlrSF   4.80e11     0.0529
7 YearBuilt   4.72e11     0.0520
8 ExterQual   4.62e11     0.0508

In [715]:
# importance proportion 
ranger(SalePrice~.,Data,importance = 'impurity')  -> rng
rng$variable.importance |> 
sort(decreasing = T) |> 
tidy() |> 
mutate(prop=x/sum(x)) |> 
#print(n=5) 
filter(prop>=0.05) |> 
select(names) |> 
unlist() -> features

Warning message:
In tidy.numeric(sort(rng$variable.importance, decreasing = T)) :
  'tidy.numeric' is deprecated.
See help("Deprecated")


In [716]:
features

       names1        names2        names3        names4        names5 
"OverallQual"   "GrLivArea"  "GarageCars" "TotalBsmtSF"   "YearBuilt" 
       names6        names7        names8 
  "ExterQual"   "X1stFlrSF"  "GarageArea" 

In [717]:

Data1 <- cbind(Data[features],SalePrice=Data$SalePrice)


In [718]:
rm(Data)

In [719]:
attach(Data1)

The following objects are masked from Data1 (pos = 3):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, YearBuilt

The following objects are masked from Data1 (pos = 4):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 5):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 6):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 7):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data (pos = 8):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF, YearBuilt

The following objects are masked from Data

The following objects are masked from Data1 (pos = 18):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 19):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 20):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 21):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 22):

    ExterQual, GarageArea, GarageCars, GrLivArea, OverallQual,
    SalePrice, TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 23):

    GarageArea, GarageCars, GrLivArea, OverallQual, SalePrice,
    TotalBsmtSF, X1stFlrSF

The following objects are masked from Data1 (pos = 24):



NULL

In [721]:
# ggplot object
Data1gg <- ggplot(Data1)

In [722]:
Data1$GarageCars |>  
  table()


  0   1   2   3   4 
 81 369 824 181   5 

In [723]:
Data1gg +
  geom_bar(aes(GarageCars))

In [724]:
Data1$YearBuilt |> 
  table() 


1872 1875 1880 1882 1885 1890 1892 1893 1898 1900 1904 1905 1906 1908 1910 
   1    1    4    1    2    2    2    1    1   10    1    1    1    2   17 
1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 
   1    3    1    7   10    8    1    7    3   30    6    8    7    7   16 
1926 1927 1928 1929 1930 1931 1932 1934 1935 1936 1937 1938 1939 1940 1941 
   9    3    7    4    9    6    4    3    6    9    5    4    8   18   15 
1942 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 
   2    6    7    5   14   12   20    6    5   12   24   16   14   20   24 
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 
  26   17   14   19   16   15   24   18   16   22   14   24   22   23   11 
1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 
  10    8   33   32   16    9   10    5    6    4    9    5    5    3   11 
1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 
   3   12  

In [725]:
Data1gg +
  geom_histogram(aes(YearBuilt))

`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.

In [726]:
table(Data1$OverallQual)


  1   2   3   4   5   6   7   8   9  10 
  2   3  20 116 397 374 319 168  43  18 

In [727]:
Data1gg +
  geom_bar(aes(factor(OverallQual)))


In [728]:
Data1$TotalBsmtSF |> 
  hist()

## finding relations

In [729]:
Data1gg +
  geom_point(aes(TotalBsmtSF,SalePrice) ) +
  geom_smooth(aes(TotalBsmtSF,SalePrice) )

`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'

In [730]:
Data1gg +
geom_point(aes(y=SalePrice,x=YearBuilt)) +
geom_smooth(aes(y=SalePrice,x=YearBuilt),se = F)

`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'

In [731]:
Data1gg +
  geom_boxplot(aes(factor(OverallQual),SalePrice) )

In [733]:
Data1gg +
  geom_boxplot(aes(factor(GarageCars),SalePrice) )

to be continued 😉!